# 4.9 Task Part 1

## 1. Import libraries and data

In [7]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [9]:
path = r'/Users/test/Desktop/Data Analysis/11-2024 Instacart Basket Analysis'

In [11]:
# Import dataframe
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

## 2. Wrangling

In [14]:
# Preview the new dataframe
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [16]:
df.shape

(206209, 10)

In [18]:
# Check data types
df.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [20]:
# Rename improper columns
df = df.rename(columns={ 
    'Surnam' : 'surname',
    'First Name' : 'first_name',
    'Gender' : 'gender',
    'Age' : 'age',
    'STATE' : 'state', 
    'fam_status' : 'family_status'})

In [22]:
df.head()

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 3. Data Consistency Checks

In [25]:
# Check mixed data-types in columns
for col in df.columns.tolist():
  weird = (df[[col]].map(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)

first_name


In [27]:
# The 'first_name' column has mixed data-types that need to be addressed
df['first_name'] = df['first_name'].astype('str')

In [29]:
# The data type of 'user_id' column is improperly listed as an integer, which needs to be changed.
df['user_id'] = df['user_id'].astype('str')

In [31]:
# Recheck the data-types
df.dtypes

user_id          object
first_name       object
surname          object
gender           object
state            object
age               int64
date_joined      object
n_dependants      int64
family_status    object
income            int64
dtype: object

In [33]:
# Check for missing values
df.isnull().sum()

user_id          0
first_name       0
surname          0
gender           0
state            0
age              0
date_joined      0
n_dependants     0
family_status    0
income           0
dtype: int64

In [35]:
# Check for duplicates
df_dups = df[df.duplicated()]

In [37]:
df_dups

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,family_status,income


Everything looks fine.

## 4. Combining dataframes

In [41]:
# Import the rest of the prepared data
orders_products = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_grouped.pkl'))

In [43]:
# Check the shape of the imported dataframe
orders_products.shape

(32399732, 25)

In [45]:
# Check the columns
orders_products.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend,spending_flag,customer_frequency,order_frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Mid-range product,Regularly busy,Regularly busy,Most orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Mid-range product,Regularly busy,Regularly busy,Average orders,32,Regular customer,6.935811,Low spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,7.930208,Low spender,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Mid-range product,Regularly busy,Slowest days,Most orders,3,New customer,4.972414,Low spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Mid-range product,Least busy,Slowest days,Average orders,3,New customer,4.972414,Low spender,9.0,Frequent customer


In [47]:
# Both dataframes shared 'user_id' column. Check the datatypes of them.
orders_products.dtypes['user_id']

dtype('int64')

In [49]:
df.dtypes['user_id']

dtype('O')

In [51]:
# Change the data type of 'user_id' in orders_products dataframe
orders_products['user_id'] = orders_products['user_id'].astype('str')

In [53]:
# Drop the '_merge' column which is unnecessary.
orders_products = orders_products.drop(columns=['_merge'])

In [55]:
# Merge the two dataframes using the column 'user_id' as the key column
custs_ords_prods = orders_products.merge(df, on = 'user_id', indicator = True)

In [57]:
# Check shape of merged dataframe
custs_ords_prods.shape

(32399732, 34)

In [59]:
custs_ords_prods.head(5)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,first_name,surname,gender,state,age,date_joined,n_dependants,family_status,income,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620,both
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Deborah,Glass,Female,Vermont,66,6/16/2018,2,married,158302,both
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Heather,Myers,Female,Wisconsin,40,2/9/2020,3,married,31308,both
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Heather,Myers,Female,Wisconsin,40,2/9/2020,3,married,31308,both


In [61]:
# Check inner join value counts
custs_ords_prods['_merge'].value_counts()

_merge
both          32399732
left_only            0
right_only           0
Name: count, dtype: int64

In [63]:
# Check data-types
custs_ords_prods.dtypes

product_id                   int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
order_id                     int64
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
first_order                   bool
add_to_cart_order            int64
reordered                    int64
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_spend              float64
spending_flag               object
customer_frequency         float64
order_frequency_flag        object
first_name                  object
surname                     object
gender                      object
state                       object
age                 

In [65]:
# Drop the new '_merge' column. We don't need it anymore.
custs_ords_prods = custs_ords_prods.drop(columns=['_merge'])

## 5. Export this new dataframe as a pickle file.

In [68]:
custs_ords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged.pkl'))